# Investigation: AWS Application Load Balancer Insecure SSL Policy

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type="aws:cloudtrail"
eventSource="elasticloadbalancing.amazonaws.com"
eventName:(
  "ModifyListener"
  "CreateListener"
)
and requestParameters.sslPolicy:(
  "ELBSecurityPolicy-2016-08" or
  "ELBSecurityPolicy-TLS13-1-1-2021-06" or
  "ELBSecurityPolicy-TLS13-1-0-2021-06" or
  "ELBSecurityPolicy-TLS13-1-1-FIPS-2023-04" or
  "ELBSecurityPolicy-TLS13-1-0-FIPS-2023-04" or
  "ELBSecurityPolicy-TLS-1-1-2017-01" or
  "ELBSecurityPolicy-FS-2018-06" or
  "ELBSecurityPolicy-2015-05" or
  "ELBSecurityPolicy-TLS-1-0-2015-04" or
  "ELBSecurityPolicy-FS-1-1-2019-08"
)
and not errorCode:*
| stats
    min(timestamp) as firstTime,
    max(timestamp) as lastTime,
    count() as eventCount
  by
    recipientAccountId,
    userIdentity.arn,
    eventSource,
    eventName,
    awsRegion,
    requestParameters.sslPolicy
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()